# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.129/12451/1 Dashboard: http://10.20.0.129:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1026,George,-0.773794155861713,0.5098303842588172
2000-01-01 00:00:01,960,Jerry,0.19645951020113372,0.5926706934767549
2000-01-01 00:00:02,1060,Sarah,-0.07062890194551064,0.6210674504784142
2000-01-01 00:00:03,987,Tim,0.9434422411668777,0.21926304777823247
2000-01-01 00:00:04,992,Ingrid,0.19835956168243052,-0.6449520020486252
2000-01-01 00:00:05,1043,Sarah,0.048086421405284296,0.9313557900608995
2000-01-01 00:00:06,1004,Quinn,0.7328986403114914,-0.2493326546546022
2000-01-01 00:00:07,974,Bob,-0.27108514190373456,-0.34073353996889644
2000-01-01 00:00:08,984,Laura,-0.6865568853264938,-0.6859336747247433


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,978,Yvonne,0.4642969870639986,0.7488428953089796
2000-01-30 00:00:01,1016,Tim,0.8683448010137098,0.7771465449039086
2000-01-30 00:00:02,938,Oliver,-0.7201794468213383,0.4575258829413684
2000-01-30 00:00:03,1004,Wendy,-0.9403205855682262,0.6865325049015412
2000-01-30 00:00:04,1025,Jerry,-0.7551924409771325,0.15598550610225748
2000-01-30 00:00:05,1015,Wendy,0.47214082244126887,0.7570131229800909
2000-01-30 00:00:06,990,Ingrid,-0.6146790392405979,-0.036736513402389814
2000-01-30 00:00:07,943,Quinn,-0.22885845537598137,0.882657976172919
2000-01-30 00:00:08,972,Alice,-0.03064639025043081,0.5228742969935671


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1026,George,-0.773794,0.509830
1,2000-01-01 00:00:01,960,Jerry,0.196460,0.592671
2,2000-01-01 00:00:02,1060,Sarah,-0.070629,0.621067
3,2000-01-01 00:00:03,987,Tim,0.943442,0.219263
4,2000-01-01 00:00:04,992,Ingrid,0.198360,-0.644952


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1026,George,-0.773794,0.509830
1,2000-01-01 00:00:01,960,Jerry,0.196460,0.592671
2,2000-01-01 00:00:02,1060,Sarah,-0.070629,0.621067
3,2000-01-01 00:00:03,987,Tim,0.943442,0.219263
4,2000-01-01 00:00:04,992,Ingrid,0.198360,-0.644952


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.55 s, sys: 636 ms, total: 5.19 s
Wall time: 3.41 s


name
Alice      -0.002979
Bob         0.001500
Charlie     0.000102
Dan         0.001022
Edith      -0.000515
Frank      -0.000329
George      0.002297
Hannah     -0.000076
Ingrid     -0.002415
Jerry      -0.000234
Kevin       0.001023
Laura       0.000768
Michael     0.001677
Norbert     0.001088
Oliver      0.000515
Patricia    0.003580
Quinn       0.000744
Ray        -0.001492
Sarah      -0.001730
Tim        -0.000768
Ursula     -0.002760
Victor     -0.002405
Wendy      -0.003739
Xavier     -0.001179
Yvonne      0.000672
Zelda       0.001473
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.44 s, sys: 194 ms, total: 1.63 s
Wall time: 1.1 s


name
Alice      -0.002979
Bob         0.001500
Charlie     0.000102
Dan         0.001022
Edith      -0.000515
Frank      -0.000329
George      0.002297
Hannah     -0.000076
Ingrid     -0.002415
Jerry      -0.000234
Kevin       0.001023
Laura       0.000768
Michael     0.001677
Norbert     0.001088
Oliver      0.000515
Patricia    0.003580
Quinn       0.000744
Ray        -0.001492
Sarah      -0.001730
Tim        -0.000768
Ursula     -0.002760
Victor     -0.002405
Wendy      -0.003739
Xavier     -0.001179
Yvonne      0.000672
Zelda       0.001473
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.31 s, sys: 81 ms, total: 1.39 s
Wall time: 960 ms


name
Alice      -0.002979
Bob         0.001500
Charlie     0.000102
Dan         0.001022
Edith      -0.000515
Frank      -0.000329
George      0.002297
Hannah     -0.000076
Ingrid     -0.002415
Jerry      -0.000234
Kevin       0.001023
Laura       0.000768
Michael     0.001677
Norbert     0.001088
Oliver      0.000515
Patricia    0.003580
Quinn       0.000744
Ray        -0.001492
Sarah      -0.001730
Tim        -0.000768
Ursula     -0.002760
Victor     -0.002405
Wendy      -0.003739
Xavier     -0.001179
Yvonne      0.000672
Zelda       0.001473
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html